In [1]:
import time
import math
import random
import numpy as np

import spacy
spacy.load('fr_core_news_sm')
spacy.load('en_core_web_sm')

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

In [3]:
spacy_fr = spacy.load("fr_core_news_sm")
print(len(spacy_fr.vocab))
spacy_en = spacy.load("en_core_web_sm")
print(len(spacy_en.vocab))

424
761


In [4]:
def tokenize_fr(text):
    return [tok.text for tok in spacy_fr.tokenizer(text)][::-1]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)][::-1]

In [5]:
SRC = Field(tokenize=tokenize_fr, 
            init_token="<sos>", 
            eos_token="<eos>", 
#             unk_token="<unk>",  #TODO: unk가 꼭 필요한가?
            lower=True)

TRG = Field(tokenize=tokenize_en, 
            init_token="<sos>", 
            eos_token="<eos>",
#             unk_token="<unk>",
            lower=True)

print(type(SRC), type(TRG))

<class 'torchtext.data.field.Field'> <class 'torchtext.data.field.Field'>


/home/soohyun/anaconda3/envs/torch/lib/python3.9/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [6]:
train_data, valid_data, test_data = Multi30k.splits(path="/home/soohyun/data/multi30k/",\
                                                    exts=(".fr",".en"), fields=(SRC, TRG))

print('train:',len(train_data))
print('valid:',len(valid_data))
print('test:',len(test_data))

/home/soohyun/anaconda3/envs/torch/lib/python3.9/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


train: 29000
valid: 1014
test: 1000


In [7]:
print(vars(train_data.examples[0])) 

{'src': ['.', 'buissons', 'de', 'près', 'dehors', 'sont', 'blancs', 'hommes', 'jeunes', 'deux'], 'trg': ['.', 'bushes', 'many', 'near', 'outside', 'are', 'males', 'white', ',', 'young', 'two']}


In [8]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [9]:
print('len of French vocab:',len(SRC.vocab))
print('len of English vocab:',len(TRG.vocab))

len of French vocab: 6470
len of English vocab: 5893


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

device: cuda


In [11]:
BATCH_SIZE = 50
# https://github.com/karlie38/torchstudy/blob/master/2.%20Sequence%20to%20Sequence%20Learning%20wiht%20Neural%20Networks/seq2seq-v03.ipynb
train_iter, valid_iter, test_iter = BucketIterator.splits((train_data, valid_data, test_data),
                                                         batch_size=BATCH_SIZE,
                                                         device=device,
                                                         sort_key=lambda x:len(x.src))
                                                        # src 문장을 길이로 sort

print(len(train_iter), len(valid_iter), len(test_iter))

580 21 20


/home/soohyun/anaconda3/envs/torch/lib/python3.9/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [42]:
#TODO:::: https://github.com/karlie38/torchstudy/blob/master/2.%20Sequence%20to%20Sequence%20Learning%20wiht%20Neural%20Networks/seq2seq-v03.ipynb
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
#         print('[enc]hid dim:', hid_dim)
        self.hid_dim = hid_dim
        self.n_layers = n_layers
#         print('[enc]n layers:', n_layers)
        self.embedding = nn.Embedding(input_dim, emb_dim)
#         print('[enc]embedding (input_dim, emb_dim):', (input_dim, emb_dim))
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
#         print('[enc]LSTM(input_dim, emb_dim, n layers):', (input_dim, emb_dim, n_layers))
#         print('[enc]dropout:', dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        src_ = src.transpose(1,0)
        input_lengths = torch.LongTensor([torch.max(src_[i,:].data.nonzero())+1 for i in range(src_.size(0))])
        input_lengths, sorted_idx = input_lengths.sort(0, descending=True)
        input_seq2idx = src_[sorted_idx]
        
        embedded = self.dropout(self.embedding(src))
        embedded = embedded.transpose(1,0)
        
        packed_input = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths.tolist(), batch_first=True)
        packed_output, (hidden, cell) = self.rnn(packed_input)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        
        output = output.transpose(1,0)

        return output, hidden, cell

In [43]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        if input.dim() == 2:
            input = input
        else:
            input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

In [44]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert encoder.hid_dim == decoder.hid_dim, \
        "hidden dimensions of encoder and decoder must be equal"
        assert encoder.n_layers == decoder.n_layers, \
        "encoder and decoder must have equal number of layers"
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        # last hidden state of the encoder is used as the init hidden state of the decoder
        encoder_output, hidden, cell = self.encoder(src)
        
        # first input to the decoder is the <sos> token
        input = trg[0,:]
        
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            # receive output tensor (predictions) and new hidden and cell states
            outputs[t] = output
            
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            # get the highest predicted token from out predictions
            top1 = output.argmax(1)
            
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            
        return outputs

In [45]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [46]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(6470, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [47]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model)} trainable parameters")

The model has 13544453 trainable parameters


In [48]:
optimizer = optim.Adam(model.parameters())

In [49]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [50]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        output = model(src, trg)
        
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        epoch_loss += loss.item()
    return epoch_loss/len(iterator)

In [51]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for i,batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            
            output = model(src, trg, 0)  # turn off teacher forcing
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            loss = criterion(output, trg)
            epoch_loss += loss.item()
            
    return epoch_loss/len(iterator)

In [52]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time/60)
    elapsed_secs = int(elapsed_time - (elapsed_mins *60))
    return elapsed_mins, elapsed_secs

In [53]:
N_EPOCHS = 1
CLIP = 1

best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)
    print('valid loss:', valid_loss)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "fr2en.pt")
        
    print(f"Epoch: {epoch+1:2} | Time: {epoch_mins}m {epoch_secs}s")
    print(f"\tTrain loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}")
    print(f"\tVal. loss: {valid_loss:.3f} | Val. PPL: {math.exp(valid_loss):7.3f}")

valid loss: 4.931637695857456
Epoch:  1 | Time: 1m 54s
	Train loss: 4.743 | Train PPL: 114.832
	Val. loss: 4.932 | Val. PPL: 138.606


In [54]:
model.load_state_dict(torch.load('fr2en.pt'))
test_loss = evaluate(model, test_iter, criterion)
print(f"Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f}")

Test Loss: 4.902 | Test PPL: 134.496


In [58]:
# BLEI score:https://github.com/karlie38/torchstudy/blob/master/2.%20Sequence%20to%20Sequence%20Learning%20wiht%20Neural%20Networks/seq2seq-v03.ipynb

from torchtext.data.metrics import bleu_score
from tqdm import tqdm

In [65]:
#TODO: 내 원래 코드에서의 max_length가 몇이지? 그런거 업ㅂ따 
def translate_sentence(model, sentence, src_field, trg_field, device, max_length=50):
    if type(sentence) == str:
        tokens = tokenize_en(sentence)
    else:
        tokens = [token.lower() for token in sentence]
        
    tokens.insert(0, src_field.init_token)
    tokens.append(src_field.eos_token)
    
    text_to_indices = [src_field.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)
    
    # build encoder hidden, cell state
    with torch.no_grad():
        output, hidden, cell = model.encoder(sentence_tensor)

    outputs = [trg_field.vocab.stoi['<sos>']]
    
    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)
        
#         print('previous word:', previous_word.size())
#         print('hidden:', hidden.size())
#         print('cell:', cell.size())
        with torch.no_grad():
            output, hidden, cell = model.decoder(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()
            
        outputs.append(best_guess)
        
        # model predicts it's the end of the sentence
        if output.argmax(1).item() == trg_field.vocab.stoi['<eos>']:
            break
            
    translated_sentence = [trg_field.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:-1]

In [66]:
label = []
predict = []

for i in tqdm(range(len(test_data.examples))):
    sentence = vars(test_data.examples[i])['src']
    label.append([vars(test_data.examples[i])['trg']])
    predict.append(translate_sentence(model, sentence, SRC, TRG, device, max_length=50))
    
print('finish!')
print(f"BLEU SCORE: {bleu_score(predict, label) * 100:.2f}%")

100%|██████████| 1000/1000 [00:10<00:00, 92.96it/s]


finish!
BLEU SCORE: 2.71%


reference : https://docs.chainer.org/en/stable/examples/seq2seq.html